In [1]:
# import the right modules
import requests
import pandas as pd

In [13]:
# define the endpoints
endpoint_typed_dataset = "https://opendata.cbs.nl/ODataApi/odata/83739ENG/TypedDataSet"
endpoint_household_characteristics = "https://opendata.cbs.nl/ODataApi/odata/83739ENG/HouseholdCharacteristics"

In [15]:
# execute the request
response_typed_dateset = requests.get(endpoint_typed_dataset)
response_household_characteristics = requests.get(endpoint_household_characteristics)

In [16]:
# get the data
data_typed_dataset = response_typed_dateset.json()
data_household_characteristics = response_household_characteristics.json()

In [18]:
# convert the data to a pandas DataFrame
df_typed_dateset =  pd.DataFrame(data_typed_dataset['value'])
df_household_characteristics = pd.DataFrame(data_household_characteristics['value'])

In [20]:
# show the data of the typed dataset
df_typed_dateset.head()

,ID,HouseholdCharacteristics,Periods,NumberOfHouseholds_1,MeanDisposableIncome_2,MeanEquivalisedIncome_3,MeanExpenditures_4,MedianWealth_5,LowIncomeHouseholds_6,IncomeInequality_7,WealthInequality_8
0,0,1050010,2011JJ00,7347.6,36.6,25.6,NaN,31.8,6.9,0.286,0.777
1,1,1050010,2012JJ00,7412.1,36.8,25.8,NaN,25.1,8.0,0.288,0.791
2,2,1050010,2013JJ00,7467.8,37.0,25.9,NaN,17.2,8.9,0.288,0.818
3,3,1050010,2014JJ00,7496.4,39.0,27.3,NaN,17.3,8.5,0.302,0.818
4,4,1050010,2015JJ00,7568.5,38.6,27.1,33.8,20.9,8.2,0.288,0.814


In [21]:
# show the data of the characteristics
df_household_characteristics.head()

,Key,Title,Description,CategoryGroupID
0,1050010,Private households,One or more persons sharing the same living sp...,1
1,1050015,Type: Single person household,Private household consisting of one person.,2
2,1016010,Type: Multi-person household,Private household consisting of two or more pe...,2
3,1050190,Type: One-parent family,Private household consisting of one parent and...,2
4,1050055,"Type: Couple, total",Private household consisting of a couple and o...,2


In [26]:
# join the tables
df_joined = df_typed_dateset.merge(df_household_characteristics, 
                                   left_on="HouseholdCharacteristics",
                                   right_on="Key",
                                   how="left")

# show the result
df_joined

,ID,HouseholdCharacteristics,Periods,NumberOfHouseholds_1,MeanDisposableIncome_2,MeanEquivalisedIncome_3,MeanExpenditures_4,MedianWealth_5,LowIncomeHouseholds_6,IncomeInequality_7,WealthInequality_8,Key,Title,Description,CategoryGroupID
0,0,1050010,2011JJ00,7347.6,36.6,25.6,NaN,31.8,6.9,0.286,0.777,1050010,Private households,One or more persons sharing the same living sp...,1
1,1,1050010,2012JJ00,7412.1,36.8,25.8,NaN,25.1,8.0,0.288,0.791,1050010,Private households,One or more persons sharing the same living sp...,1
2,2,1050010,2013JJ00,7467.8,37.0,25.9,NaN,17.2,8.9,0.288,0.818,1050010,Private households,One or more persons sharing the same living sp...,1
3,3,1050010,2014JJ00,7496.4,39.0,27.3,NaN,17.3,8.5,0.302,0.818,1050010,Private households,One or more persons sharing the same living sp...,1
4,4,1050010,2015JJ00,7568.5,38.6,27.1,33.8,20.9,8.2,0.288,0.814,1050010,Private households,One or more persons sharing the same living sp...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,451,1020960,2018JJ00,776.1,105.3,68.9,NaN,283.2,0.0,0.312,0.725,1020960,Equivalised income: 10th 10%-group,Equivalised income equals disposable income co...,7
452,452,1020960,2019JJ00,782.7,121.9,79.5,NaN,326.4,0.0,0.314,0.708,1020960,Equivalised income: 10th 10%-group,Equivalised income equals disposable income co...,7
453,453,1020960,2020JJ00,789.4,115.9,75.4,56.6,344.1,0.0,0.368,0.704,1020960,Equivalised income: 10th 10%-group,Equivalised income equals disposable income co...,7
454,454,1020960,2021JJ00,795.1,121.4,79.1,NaN,403.7,0.0,0.437,0.666,1020960,Equivalised income: 10th 10%-group,Equivalised income equals disposable income co...,7


In [40]:
# group the data
df_grouped = (
    df_joined
    .groupby(['Periods'])
    .agg({"NumberOfHouseholds_1": "sum", 
          "MeanDisposableIncome_2": "mean"})
    .reset_index()
    .assign(Year = lambda x: x['Periods'].str.split("JJ").str[0])
    .sort_values(by="Year")
)

# show the result
df_grouped['delta'] = df_grouped['MeanDisposableIncome_2'].diff(periods=1)
df_grouped

,Periods,NumberOfHouseholds_1,MeanDisposableIncome_2,Year,delta
0,2011JJ00,68558.3,33.623684,2011,NaN
1,2012JJ00,69179.9,33.752632,2012,0.128947
2,2013JJ00,69765.7,33.905263,2013,0.152632
3,2014JJ00,70039.8,35.813158,2014,1.907895
4,2015JJ00,70618.3,35.457895,2015,-0.355263
5,2016JJ00,71034.2,36.873684,2016,1.415789
6,2017JJ00,71612.0,38.228947,2017,1.355263
7,2018JJ00,72064.7,39.121053,2018,0.892105
8,2019JJ00,72585.9,42.073684,2019,2.952632
9,2020JJ00,73256.5,42.710526,2020,0.636842
